### Stratégie :
On récupère le point de départ, on le région growth.
On fait une recherche sur la couche du dessous sur le centre trouvé précédemment puis en région growing sur les plus proches voisins jusqu'à trouver un point d'intensité similaire puis on le region growth. On limite la recherche de zone sur la couche suivante à la taille de la tache trouvée sur la précédente couche + 1 voisin

In [1]:

import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import cv2
from tqdm.notebook import tqdm
from napari.utils.colormaps import colormap_utils as cu
from functools import partial
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from networkclass import *
from functions.segmentation_utils import *

In [2]:
image_array = np.load(Path("temp/preprocessed_image.npy"))

# We extract the aorta position on the first slice of the array
target = np.array(extract_centroid_first_slice(image_array)).astype(np.uint16)
network_engine = NetworkEngine(image_array,len(image_array)-1)
# We initialise the engine with the aorta position/intesity
network_engine.network_manager.set_branch_target(0,target)
network_engine.network_manager.get_nested_intensity(0).add_number(image_array[len(image_array)-1,target[1],target[0]])
network_engine.network_manager.get_mean_radius(0).add_number(10)
# We run the engine which will try to recreate the network going from chest to feet
network_engine.run()
#This is not enough efficient to find all the arteries and we usually miss some of the main ones

#We identify that it's easy to segment knee artery, so we segment knee to then get the knee artery. We then send a exploration to the chest and to the feet

knee_depth, knee_contour = find_knee_depth(image_array)
vessel1,vessel2 = extract_arteries_position_from_knee(image_array[knee_depth],knee_contour)

network_engine.explore_reverse(vessel1,knee_depth)
network_engine.explore_reverse(vessel2,knee_depth)
network_engine.force_prepare_explore(vessel1,knee_depth)
network_engine.force_prepare_explore(vessel2,knee_depth)
network_engine.run()

  0%|          | 0/1331 [00:00<?, ?it/s]

Splitted 0 because protected until 1531
Found target is (248, 232) radius is 19.163867950439453
Create branch id :1
SUCCESS SPLIT PARENT 0
Splitted 1 because protected until 1395
Found target is (244, 165) radius is 5.918512344360352
Create branch id :2
Splitted 1 because protected until 1395
Found target is (246, 166) radius is 4.743516445159912
Create branch id :3
Splitted 0 because protected until 1375
Found target is (252, 217) radius is 11.72016429901123
Create branch id :4
SUCCESS SPLIT PARENT 0
Splitted 4 because protected until 1319
Found target is (264, 208) radius is 6.072520732879639
Create branch id :5
SUCCESS SPLIT PARENT 4
Splitted 5 because protected until 1263
Found target is (219, 224) radius is 5.402753829956055
Create branch id :6
SUCCESS SPLIT PARENT 5
Splitted 4 because protected until 1243
Found target is (300, 216) radius is 5.54775857925415
Create branch id :7
SUCCESS SPLIT PARENT 4
Splitted 7 because protected until 1154
Found target is (297, 245) radius is 4.6

In [3]:

import gzip
import napari

# We display the raw segmented image

mask_list = network_engine.network_manager.generate3DImages(image_array.shape)

SHOW_IN_NAPARI = False

if SHOW_IN_NAPARI:
    viewer = napari.Viewer()
    image_layer = viewer.add_image(image_array)
    colorindex = 0
    for mask in mask_list:
        image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex%5],blending="additive")
        colorindex+=1
        print(np.sum(mask))
    
    points_layer = viewer.add_points(
    network_engine.network_manager.label_coord,
    text=network_engine.network_manager.label
    )


    # start the event loop and show the viewer
    napari.run()



branch 0 start 1530 length 230
branch 1 start 1395 length 30
branch 4 start 1319 length 193
branch 5 start 1263 length 150
branch 6 start 1166 length 94
branch 7 start 1154 length 20
branch 8 start 1134 length 39
branch 20 start 1083 length 65
branch 21 start 1040 length 320
branch 22 start 1019 length 426
branch 23 start 970 length 377
branch 24 start 720 length 201
branch 25 start 521 length 322
branch 26 start 1530 length 975
branch 27 start 1530 length 975
branch 28 start 556 length 36
branch 29 start 556 length 50
branch 30 start 520 length 320
branch 31 start 507 length 307


In [4]:
# Network merging and cleaning

network_engine.network_manager.sanitize()

# Merge network is not optimised, running it two times make it better (in our test image)
merged_network_manager = network_engine.merge_network(network_engine.network_manager)
second_merged_network_manager = network_engine.merge_network(merged_network_manager)

# Segmentize create one branch for each vessel (1 parent gives 2 branch)
segmented_network_manager = network_engine.segmentize(second_merged_network_manager)

mask_list = network_engine.network_manager.generate3DImages(image_array.shape)
mask_list_merged = segmented_network_manager.generate3DImages(image_array.shape)

Sanitize : remove 22 (predicted)
Sanitize : remove 23 (predicted)
Sanitize : remove 31 (predicted)
0
Can't find parent for 1 at depth 1395
0
Can't find parent for 26 at depth 1300
26
at depth 1258 create 2 new branch for parent & child
27 is now 3
26 is now 4
26
at depth 1166 create 2 new branch for parent & child
5 is now 5
26 is now 6
27
at depth 1154 create 2 new branch for parent & child
7 is now 7
27 is now 8
7
Parent not present at current depth for 8 at depth 1134
False
Can't find parent for 4 at depth 1132
26
at depth 1092 create 2 new branch for parent & child
6 is now 10
26 is now 11
26
at depth 1031 create 2 new branch for parent & child
20 is now 12
26 is now 13
26
Parent not present at current depth for 24 at depth 555
27
Parent not present at current depth for 29 at depth 555
24
Parent not present at current depth for 25 at depth 519
branch 0 start 1530 length 230
branch 1 start 1395 length 30
branch 4 start 1319 length 193
branch 5 start 1263 length 150
branch 6 start 11

In [5]:
import vispy

#TODO Correct merge problem at 1320

SHOW_IN_NAPARI = False 
SAVE_IN_FILE = True


if SHOW_IN_NAPARI:
    #custom = vispy.color.Colormap([[0.0, 0.0, 0.0],[1.0, 0.0, 0.0],[1.0, 0.37, 0.12],[1.0, 1.0, 0.0], [0.0, 1.0, 0.0]],controls=(0,0.003,0.33,0.66,1.0))
    viewer = napari.Viewer()
    image_layer = viewer.add_image(image_array)
    colorindex = 0
    for mask in mask_list_merged:
        if np.sum(mask) > 0:
            image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex%5],blending="additive")
            colorindex+=1

    # start the event loop and show the viewer
    napari.run()

if SHOW_IN_NAPARI:
    #custom = vispy.color.Colormap([[0.0, 0.0, 0.0],[1.0, 0.0, 0.0],[1.0, 0.37, 0.12],[1.0, 1.0, 0.0], [0.0, 1.0, 0.0]],controls=(0,0.003,0.33,0.66,1.0))
    viewer = napari.Viewer()
    image_layer = viewer.add_image(image_array)
    colorindex = 0
    for mask in mask_list:
        if np.sum(mask) > 0:
            image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex%5],blending="additive")
            colorindex+=1

    # start the event loop and show the viewer
    napari.run()

for mask in mask_list_merged:
    print(np.sum(mask))
    
if SAVE_IN_FILE:
    f = gzip.GzipFile("temp/mask_list.npy.gz", "w")
    np.save(file=f, arr= mask_list_merged[:-1])# We remove debug mask
    f.close()



5079345
250155
795855
1145970
1033770
305490
524025
409530
3022515
64005
158100
673455
80835
3156390
0
